In [193]:
import json
import os
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

In [194]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = "cuda"

In [195]:
with open("/raid/speech/sabyasachi/Translation/COLD_decoding/data/commongen/commongen.dev.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

sentences = []
for i in range(len(data)):

    sentences.append(data[i]["scene"])

In [196]:
start = 0
end = 6
sentences = sentences[start:end]
# sentences.to(device)

In [197]:
with open("/raid/speech/sabyasachi/Translation/COLD_decoding/data/commongen/topk_changes/0f_zx_seed12_0_5_lexical_generation_cw0.500_c2w0.100_lrnllp0.600_len10_topk2_niter2000_frozlen0_winiter1000_noiseiter1_gsstd0.0100_lr0.100_lrratio1.00_lriter1000_50,500,1000,1500_1,0.5,0.1,0.05_output.json") as f:
    pred_data = [json.loads(line) for line in f]

In [198]:
pred_sentences = []

for i in range(len(pred_data)):
    pred_sentences.append(pred_data[i]["generation_complete"])

In [199]:
model = GPT2LMHeadModel.from_pretrained(
    'gpt2-xl', output_hidden_states=True,
    resid_pdrop=0, embd_pdrop=0, attn_pdrop=0, summary_first_dropout=0)

In [200]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')

In [201]:
PPL_avg = []

In [202]:
# PPL_generated = {}

In [203]:
for i in range(len(pred_sentences)):
    PPL = []
    for j in range(5):
        inputs = tokenizer(pred_sentences[i][j], return_tensors = "pt")
        loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
        ppl = torch.exp(loss).item()
        # PPL_generated[pred_sentences[i][j]] = np.round(ppl, 2)
        PPL.append(ppl)
    PPL_avg.append(np.mean(PPL))

In [204]:
# json_gen = json.dumps(PPL_generated, indent=4)

In [205]:
# with open("perplexity_gen.json", "w") as outfile:
#     outfile.write(json_gen)

In [206]:
PPL_avg_orig = []

In [207]:
# PPL_ref = {}

In [208]:
for i in range(len(sentences)):
    # PPL = []
    for j in range(len(sentences[i])):
        inputs = tokenizer(sentences[i][j], return_tensors = "pt")
        loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
        ppl = torch.exp(loss).item()
        # PPL_ref[sentences[i][j]] = np.round(ppl, 2)
        PPL.append(ppl)
    PPL_avg_orig.append(np.mean(PPL))

In [209]:
# json_ref = json.dumps(PPL_ref, indent=4)

In [210]:
# with open("perplexity_ref.json", "w") as outfile:
#     outfile.write(json_ref)

In [211]:
print("generated", np.mean(PPL_avg))
print("reference", np.mean(PPL_avg_orig))

generated 28.580418157577515
reference 104.51772686677252
